# Voronoi Diagrams

Vornoi Diagrams are a powerful computational geometry tool for analysing metric spaces. They are used to understand how KNN algorithm works. They are also used in manifold learning. They are also used in tradtional AI based planners for navigation.

## Reference

*[Notes on Vornoi Diagrams](https://cs.brown.edu/courses/cs252/misc/resources/lectures/pdf/notes09.pdf)

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import math
%matplotlib qt

In [32]:
x= np.random.rand(100,1)
y= np.random.rand(100,1)
legend_list = []

In [33]:
def setup_plot_env():
    fig, ax = plt.subplots(figsize=(7, 7))
    np.set_printoptions(precision = 8, suppress = True)
    return fig, ax

def show_legend(fig, ax):
    #plot legend
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc="lower right")
    fig.show()
    

def forceAspect(ax, aspect=1):
    ax.set_aspect('equal')
    
def plot_random_points_pivot(fig, ax, x, y, pivot_pair_num):
    #plotting the random point matrix
    ax.scatter(x,y,c="orange", label="random points")
    #pick the first point to plot the voronoi diagram of
    ax.scatter(x[pivot_pair_num],y[pivot_pair_num],c="blue", label="pivot point")
    fig.show()
    #define a global variable to store the euclidean distance of the all the points in the set S with respect to s
    global euc_dis, midpoints, slopes
    euc_dis = [] # this is the list of euclidean distances to midpoints between pivot and random point
    midpoints = [] # this is the list of midpoints
    slopes = [] # this is the list of slopes to random points (and NOT the slope of perpendicular bisectors)
    
def comp_euc_dis(point, midpoint):
    return math.sqrt((point[0]-midpoint[0])** 2 +  (point[1]-midpoint[1])** 2)

def compute_delimiter_slope_midpoint(pivot,xy):
    midpoint = (float((pivot[0]+xy[0])/2),float((pivot[1]+xy[1])/2))
    try:
        slope = float((pivot[1] - xy[1]) / (pivot[0] - xy[0]))
    except:
        slope = nan
    return slope,midpoint

#plotting perpendicular bisectors wrt the chosen point
def plot_perpendicular_bisector(fig,ax,x, y, pivot_pair_num):
    xy_list = np.array([x,y], np.float64).T.reshape(np.size(x),2)
    #print(xy_list.shape)
    #print(xy_list)
    for xy in xy_list:
        slope,midpoint = compute_delimiter_slope_midpoint((x[pivot_pair_num],y[pivot_pair_num]),xy)
        euc_dis.append(comp_euc_dis(xy,midpoint))
        midpoints.append(midpoint)
        slopes.append(slope)
        if((xy[0] != midpoint[0]) and (xy[1] != midpoint[1])):
            ax.scatter(midpoint[0],midpoint[1], c="purple", label="midpoints")
        ax.axline(midpoint, slope=-1/slope, linestyle="solid",color = "green", linewidth=0.2,
                  label="perpendicular bisector")
    #print(euc_dis)
    forceAspect(ax,aspect=1)
    show_legend(fig, ax)


Plot the perpendicular bisectors

In [34]:
fig, ax = setup_plot_env()
pivot_pair_num = 0
plot_random_points_pivot(fig, ax, x, y, pivot_pair_num)
plot_perpendicular_bisector(fig, ax, x, y, 0)
euc_dis = np.array(euc_dis)

/home/ananth/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


### Algorithmic solution to intersection of half planes containing the origin is:
Two conditions have to be met on each iteratively chosen point to form a voronoi region. They are:
* PC2 & O must be on the same side of PB1 to ensure convexity
* choose PC2 from the midpoints list such that it is closest to PB1

to achieve the above, the following algorithm is proposed:
1. find the nearest halfplane / smallest euclidean distance to midpoint through which perpendicular bisector is drawn
2. find the next perpendicular bisector such that:
    - it passes through the midpoint which is on the same side of the halfplane as the pivot point
    - it is closest to the halfplane formed by the previous perpendicular bisector

In [ ]:
def hp_test():
    

In [35]:
def argminexp(arr,exp):
    p = np.array(arr)
    excptIndx = exp

    m = np.zeros(p.size, dtype=bool)
    m[excptIndx] = True
    a = np.ma.array(p, mask=m)
    return (np.argmin(a))
exception_list = [0]

In [41]:
#extracting the index of the closest midpoint => closest halfplane
cl_ind = argminexp(euc_dis,exception_list)
print(cl_ind)
ax.scatter(midpoints[cl_ind][0],midpoints[cl_ind][1], c="yellow", label="closest midpoint")
show_legend(fig, ax)
#find the equation of the half-plane through this point is
# y - midpoints[cl_ind][1] + (1/slopes[cl_ind])*(x-midpoints[cl_ind][0]) = 0  

def point_line_arbl(x,y, midpoints, slopes, cl_ind):
    s = y - midpoints[cl_ind][1] + (1/slopes[cl_ind])*(x-midpoints[cl_ind][0])    
    return s

#check if the pivot point is a - above / r - right | b - below / l - left of the halfplane
s=point_line_arbl(x[pivot_pair_num],y[pivot_pair_num], midpoints, slopes, cl_ind)




71


In [38]:
hptl = halfplane_test(s, midpoints)
print(hptl)

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]


In [21]:
midpoint_distance_list=[]
for i in range(len(hptl)):
    if hptl[i] == 1:
        midpoint_distance_list.append(euc_dis[i])
    else:
        midpoint_distance_list.append(1000)
print(midpoint_distance_list)

[0.0, 0.07348517039640337, 1000, 1000, 1000, 1000, 0.16475815348930706, 1000, 1000, 1000, 0.28761595745095403, 0.25872642007002583, 0.1512537795933817, 0.04575509307867922, 1000, 0.2252937331700873, 1000, 1000, 1000, 1000, 0.13088936469946003, 1000, 1000, 0.28071616028427177, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0.20180466094648244, 0.24152476636735654, 1000, 0.14586789347865217, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0.1167737033332306, 0.2767165563092191, 1000, 1000, 1000, 1000, 0.03692321734401912, 1000, 1000, 0.23745186924197564, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0.1862973111569127, 0.20223814844585364, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0.26344384502873075, 1000, 0.10923478631835354, 0.09326440310299135, 1000, 0.09775537111192882, 1000, 1000, 0.1809030055597164, 0.0995126919958163, 1000, 0.18360381794345798, 0.24433816299666775, 1000, 1000, 1000, 0.13188053180264175, 0.12148330381115545, 1000, 1000, 1000,

In [23]:
exception_list = [0]
argminexp(midpoint_distance_list,exception_list)

53

In [24]:
plt.scatter(midpoints[53][0], midpoints[53][1], c="black")

In [11]:
#extracting the index of the closest midpoint => closest halfplane
cl_ind = 53
ax.scatter(midpoints[cl_ind][0],midpoints[cl_ind][1], c="yellow", label="closest midpoint")
#check if the pivot point is a - above / r - right | b - below / l - left of the halfplane
s=point_line_arbl(x[pivot_pair_num],y[pivot_pair_num], midpoints, slopes, cl_ind)



In [13]:
hptl = halfplane_test(s, midpoints)
print(hptl)

[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0]


In [14]:
midpoint_distance_list=[]
for i in range(len(hptl)):
    if hptl[i] == 1:
        midpoint_distance_list.append(euc_dis[i])
    else:
        midpoint_distance_list.append(1000)
print(midpoint_distance_list)

[0.0, 0.07348517039640337, 0.30095891172111, 0.15452702122291004, 0.10640914755747942, 1000, 1000, 0.14981693715102284, 1000, 0.3372600437527556, 1000, 1000, 1000, 0.04575509307867922, 1000, 1000, 0.0949368839789198, 0.3629934866744924, 1000, 1000, 1000, 1000, 0.15600717391373684, 1000, 0.39316755866092024, 0.26047910779827904, 0.3970618562183037, 0.3825174119113422, 1000, 0.31066340183785707, 0.3489057666580028, 0.08879893128609752, 0.2812473492008388, 1000, 0.2319501569052244, 1000, 1000, 1000, 0.24017180616248576, 1000, 0.25713388522891867, 0.4411766721399112, 0.1413039583971753, 1000, 0.453441428965298, 0.11748871650328048, 0.05678892689453849, 1000, 1000, 1000, 1000, 0.36177306049432645, 0.20869490822592807, 1000, 0.3975450393370867, 0.16019745986581638, 1000, 0.08083944830821706, 0.20033187944177538, 1000, 0.1621754469895159, 0.3968458889321429, 1000, 0.030280078533723217, 0.2653274138251824, 1000, 1000, 1000, 0.09251082457397163, 0.13933918296977982, 0.35742153981558683, 1000, 0

In [15]:
np.argmin(midpoint_distance_list[1:])+1

63

Note that while doing the above we are not considering the case where the first point is the only closest point to the second point in which case it will return the first point itself. To avoid this a special argmin function has to be written which avoids a list of mentioned indices while returning the argmin